# Automated HTTP Service and Directory Brute Forcing Tool

- performs a network scan using nmap to identify open HTTP services on specified ports
- runs gobuster to brute force directories on the discovered HTTP services
- results are parsed and displayed for further analysis

### Run the program
python script_name.py example.com -w /path/to/wordlist.txt -o results.json

### Imported libraries

In [ ]:
import argparse
import json
import re
import subprocess

- **argparse** for parsing command-line arguments 
- **subprocess module** runs external commands
- **re module** for use of regular expressions
- **json module** for handling JSON data

### The 'run_nmap' function

In [ ]:
def run_nmap(target):
    command = ['nmap', '-p-', '--open', '-sV', target]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print(f"Error running nmap: {result.stderr}")
        return None
    return result.stdout

### Command construction
- **-p-** tells nmap to scan all TCP ports on the target
- **--open** instructs nmap to show only open ports
- **-sV** enables service version detection
    - attempts to determine the version of the services running on open ports
    - provides more detailed information about the services, such as the application name and version
- **target** specifies the IP address or hostname of the system you want to scan
    - this parameter is passed dynamically to the function

### Error handling
- **result.returncode** holds the exit status of the nmap command
    - return code of 0 typically indicates successful execution, while non-zero codes indicate errors
- **result.stderr** contains any error messages produced by nmap
    - if the command fails (i.e., returncode is not 0), the error message is printed

### Returning the output
- **result.stdout** contains the standard output of the command (i.e., the result of the nmap scan) for further processing

### The 'parse_nmap_output' function

In [ ]:
def parse_nmap_output(output):
    http_ports = []
    for line in output.splitlines():
        if re.search(r"http\b", line, re.IGNORECASE):
            port_match = re.search(r"(\d+)/tcp", line)
            if port_match:
                http_ports.append(port_match.group(1))
    return http_ports

### Initialize list
- **http_ports** initializes an empty list that will be used to store port numbers where http services are detected

### Process each line of output
- **output.splitlines()** splits the nmap scan output into individual lines
    - allows the function to process each line separately
    - is useful for iterating through the output line-by-line

### Search for http Services
- **re.search(r"http\b", line, re.IGNORECASE)** uses a regular expression to search for the substring "http" in the line
    - the \b ensures that "http" is matched as a whole word, not as part of another word (e.g., "https")
    - the re.IGNORECASE flag makes the search case-insensitive
    - if "http" is found in the line, it indicates that the line likely describes an http service

### Extract port number
- **re.search(r"(\d+)/tcp", line)** uses a regular expression to find port numbers in the line
    - (\d+) matches one or more digits (the port number), and /tcp specifies that the port is a TCP port
    - this pattern extracts the port number from lines indicating open TCP ports

### Add Port to List
- **port_match.group(1)** retrieves the port number matched by the first capturing group in the regular expression
- **append()** adds this port number to the http_ports list

### Return list of http ports
- **return http_ports** returns the list of port numbers where http services were detected

### The 'run_gobuster' function

In [ ]:
def run_gobuster(target, port, wordlist):
    url = f"http://{target}:{port}"
    result = subprocess.run(['gobuster', 'dir', '-u', url, '-w', wordlist], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print(f"Error running gobuster on {url}: {result.stderr}")
        return None
    return result.stdout

### Construct URL

- **url** constructs the URL to be scanned by gobuster using the provided target (domain or IP address) and port
    - the URL is formatted as http://target:port, where target is the hostname or IP address and port is the port number

### Execute gobuster command
- **subprocess.run** is used to execute the gobuster command in a subprocess
    - waits for the command to complete and then returns a CompletedProcess instance
- **arguments**:
    - dir: mode of operation for gobuster, indicating directory brute-forcing.
    - -u: specifies the URL to scan, in this case, it is the URL constructed from the target and port
    - url: full URL of the target to scan (e.g., http://example.com:80)
    - -w: specifies the path to the wordlist used for brute-forcing directories
    - wordlist: path to the file containing a list of directory names to be tested

- **stdout=subprocess.PIPE** redirects the standard output (stdout) of the gobuster command to a pipe
    - allows capturing the output for further processing.
- **stderr=subprocess.PIPE** redirects the standard error (stderr) of the gobuster command to a pipe
    - allows capturing any error messages.
- **text=True** ensures the output is returned as a string rather than bytes, making it easier to handle and process

### Error handling
- **result.returncode** holds the exit status of the gobuster command
    - return code of 0 indicates successful execution, while non-zero codes indicate errors
- **result.stderr** contains any error messages generated by gobuste
    - if the command fails (i.e., returncode is not 0), the error message is printed.

### Return output
- **result.stdout** contains the standard output of the gobuster command
    - includes the results of the directory brute-forcing operation, which is returned for further processing or display

### The main function

In [ ]:
def main(target, wordlist, output_file=None):
    """Main function to run the nmap scan, parse results, and run gobuster on HTTP ports."""
    nmap_output = run_nmap(target)
    if not nmap_output:
        print("Nmap scan failed.")
        return

    http_ports = parse_nmap_output(nmap_output)
    if not http_ports:
        print("No HTTP services found.")
        return

    results = {}
    for port in http_ports:
        print(f"Starting gobuster on {target}:{port}")
        gobuster_output = run_gobuster(target, port, wordlist)
        if gobuster_output:
            results[port] = gobuster_output

    if output_file:
        with open(output_file, 'w') as f:
            json.dump(results, f, indent=4)
        print(f"Results saved to {output_file}.")
    else:
        print("Scan results:")
        for port, output in results.items():
            print(f"Port {port}:\n{output}")

    return results

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Automated tool for scanning HTTP services and directory brute-forcing.")
    parser.add_argument("target", help="Target IP address or domain")
    parser.add_argument("-w", "--wordlist", required=True, help="Path to the wordlist file for Gobuster")
    parser.add_argument("-o", "--output", help="File path to save results. Leave blank to print to console.")

    args = parser.parse_args()
    main(args.target, args.wordlist, args.output)

### Run nmap scan
- **run_nmap(target)** calls the run_nmap function to perform a scan on the target
    - returns the output of the scan
- **if not nmap_output** checks if the nmap scan failed (i.e., no output was returned)
    - if so, prints an error message and exits the function

### Parse nmap output
- **parse_nmap_output(nmap_output)** calls the parse_nmap_output function to extract HTTP ports from the nmap scan output
- **if not http_ports** checks if no http ports were found
    - if so, prints a message indicating that no http services were detected and exits the function

### Run gobuster on http ports
- **results = {}** initializes an empty dictionary to store the results of the gobuster scans
- **for port in http ports** iterates over each port that was identified as having an http service
- **run_gobuster(target, port, wordlist)** calls the run_gobuster function to perform directory brute-forcing on the current http port
- **if gobuster_output** checks if the gobuster command produced output
    - if so, stores the output in the results dictionary with the port number as the key

### Save or print results
- **if output_file** checks if an output_file argument was provided
    - **with open(output_file, 'w') as f** opens the specified file in write mode
    - **json.dump(results, f, indent=4)** writes the results dictionary to the file in JSON format with an indentation of 4 spaces for readability
    - **print(f"Results saved to {output_file}.")** prints a message indicating that the results were saved to the specified file

- **else** no output_file was provided, the results are printed to the console
    - **for port, output in results.items()** iterates over each port and its corresponding gobuster output
    - **print(f"Results for {target}:{port}")** prints a header for the results of the current port
    - **print(output)** prints the gobuster output for the current port

### Return Results
- **return results** returns the results dictionary, which contains the gobuster output for each http port
    - allows further processing or inspection if needed.

### Command-line argument parsing
- **if __name__ == "__main__"** ensures the code block is executed only if the script is run directly, not when it is imported as a module in another script
- **argparse.ArgumentParser()** creates an instance of the ArgumentParser class, which is used to handle command-line arguments
    - **description** is a brief description of what the script does
        - will be displayed when the user runs the script with the --help option
- **parser.add_argument("target", ...)** defines a positional argument for the script
    - **target** is the name of the argument
    - **help** provides a description of what the argument represents
- **parser.add_argument("-w", "--wordlist", ...)** defines an optional argument with flags
    - **-w, --wordlist** are short (-w) and long (--wordlist) flags for the argument
    - **required=True** indicates that this argument is mandatory
    - **help** provides a description of what this argument does, shown in the help message
- **parser.add_argument("-o", "--output", ...)** defines another optional argument with flags
    - **-o, --output** are short (-o) and long (--output) flags for the argument
    - **help** provides a description of the argument, explaining that if this argument is not provided, results will be printed to the console instead
- **parser.parse_args()** parses the command-line arguments and returns an object containing the parsed arguments as attributes
- **main(args.target, args.wordlist, args.output)** calls the main function with the parsed arguments
    - **args.target** is the target IP address or domain
    - **args.wordlist** is the path to the gobuster wordlist file
    - **args.output** is the file path to save results, or None if results are to be printed to the console
